## 1. Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

## 2. Importando o dataset

In [2]:
try:
    df = pd.read_csv('../Datasets/train.csv')
    print("Arquivo carregado com sucesso!")
except FileNotFoundError:
    print("Erro: Arquivo 'train.csv' não encontrado. Verifique o caminho do arquivo.")
    exit()

Arquivo carregado com sucesso!


In [3]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


## 3. Limpeza e Separação

In [4]:
# Limpeza - Engenharia de Features (Cabin)

print("--- Bloco 3: Tratando 'Cabin' ---")

# 1. Preencher valores NaN na coluna 'Cabin'
df['Cabin'] = df['Cabin'].fillna('U/0/U')

# 2. Dividir a coluna 'Cabin' em 3 novas colunas
# Removi o 'try/except' para vermos o erro se ele acontecer aqui
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)

# 3. Converter 'Num' para tipo numérico (e preencher NaN se houver)
df['Num'] = pd.to_numeric(df['Num'], errors='coerce') # 'coerce' transforma erros em NaN
df['Num'] = df['Num'].fillna(df['Num'].median()) # Preenche NaN criados

print("Colunas 'Deck', 'Num' e 'Side' criadas.")
print(df[['Deck', 'Num', 'Side']].head())

--- Bloco 3: Tratando 'Cabin' ---
Colunas 'Deck', 'Num' e 'Side' criadas.
  Deck  Num Side
0    B    0    P
1    F    0    S
2    A    0    S
3    A    0    S
4    F    1    S


## 4. Limpeza - Preenchimento de valores ausentes

In [5]:
# Limpeza - Preenchendo Valores Ausentes (NaN)

print("--- Bloco 4: Preenchendo NaNs restantes ---")

# 1. Definir colunas
colunas_numericas = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Num']
colunas_categoricas = ['HomePlanet', 'CryoSleep', 'VIP', 'Deck', 'Side']

# 2. Preencher numéricas com MEDIANA
for col in colunas_numericas:
    if col in df.columns:
        mediana = df[col].median()
        df[col] = df[col].fillna(mediana)

# 3. Preencher categóricas com MODA (valor mais comum)
for col in colunas_categoricas:
    if col in df.columns:
        moda = df[col].mode()[0]
        df[col] = df[col].fillna(moda)

print("Valores Ausentes preenchidos.")

--- Bloco 4: Preenchendo NaNs restantes ---
Valores Ausentes preenchidos.


C:\Users\viras\AppData\Local\Temp\ipykernel_32160\2737623566.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(moda)


## 5. Limpeza - Enconding e Finalização

In [6]:
print("--- Bloco 5: Encoding ---")

# 1. Remover colunas inúteis
colunas_para_remover = ['PassengerId', 'Name', 'Cabin', 'Destination']
df = df.drop(columns=[col for col in colunas_para_remover if col in df.columns])
print("Colunas inúteis removidas.")

# 2. Converter Booleanos para 0/1
if 'CryoSleep' in df.columns:
    df['CryoSleep'] = df['CryoSleep'].map({True: True, False: False, 'True': True, 'False': False}).astype(bool)
if 'VIP' in df.columns:
    df['VIP'] = df['VIP'].map({True: True, False: False, 'True': True, 'False': False}).astype(bool)
df['Transported'] = df['Transported'].astype(bool)
print("Colunas booleanas convertidas.")

# 3. Aplicar One-Hot Encoding (A CORREÇÃO)
# Vamos forçar a codificação das colunas que SABEMOS que são texto,
# sem verificar 'dtype' ou qualquer outra coisa.
colunas_para_codificar = ['HomePlanet', 'Deck', 'Side']

# Filtra apenas as que realmente existem no dataframe
colunas_existentes = [col for col in colunas_para_codificar if col in df.columns]

print(f"Forçando One-Hot Encoding em: {colunas_existentes}")
df = pd.get_dummies(df, columns=colunas_existentes, drop_first=True)

print("\n--- Limpeza Concluída. Verificação Final: ---")
# A saída do .info() NÃO PODE TER NENHUM 'object'
print(df.info())

--- Bloco 5: Encoding ---
Colunas inúteis removidas.
Colunas booleanas convertidas.
Forçando One-Hot Encoding em: ['HomePlanet', 'Deck', 'Side']

--- Limpeza Concluída. Verificação Final: ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CryoSleep          8693 non-null   bool   
 1   Age                8693 non-null   float64
 2   VIP                8693 non-null   bool   
 3   RoomService        8693 non-null   float64
 4   FoodCourt          8693 non-null   float64
 5   ShoppingMall       8693 non-null   float64
 6   Spa                8693 non-null   float64
 7   VRDeck             8693 non-null   float64
 8   Transported        8693 non-null   bool   
 9   Num                8693 non-null   int64  
 10  HomePlanet_Europa  8693 non-null   bool   
 11  HomePlanet_Mars    8693 non-null   bool   
 12  Deck_B             8693 

## 6. Separando X e y

In [7]:
from sklearn.model_selection import train_test_split

print("--- Separando dados de Treino e Teste ---")
y = df['Transported']
X = df.drop(columns=['Transported'])

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dados de treino (X_treino): {X_treino.shape}")
print(f"Dados de teste (X_teste): {X_teste.shape}")

--- Separando dados de Treino e Teste ---
Dados de treino (X_treino): (6954, 21)
Dados de teste (X_teste): (1739, 21)


## 7. StandardScaler

In [8]:
#  Normalização dos Dados
from sklearn.preprocessing import StandardScaler

print("--- Iniciando Normalização ---")
scaler = StandardScaler()
scaler.fit(X_treino)

X_treino_scaled = scaler.transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

print("Dados de treino e teste normalizados com sucesso.")

--- Iniciando Normalização ---
Dados de treino e teste normalizados com sucesso.


## 8. Modelo 1: Regressão Logística

In [9]:
#  Modelo 1 - Regressão Logística

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print("--- Treinando Modelo 1: Regressão Logística ---")
modelo_lr = LogisticRegression(random_state=42, max_iter=1000)
modelo_lr.fit(X_treino_scaled, y_treino)
y_pred_lr = modelo_lr.predict(X_teste_scaled)

acuracia_lr = accuracy_score(y_teste, y_pred_lr)
print(f"\n--- Resultados da Regressão Logística ---")
print(f"Acurácia: {acuracia_lr:.4f}")
print(classification_report(y_teste, y_pred_lr))

--- Treinando Modelo 1: Regressão Logística ---

--- Resultados da Regressão Logística ---
Acurácia: 0.7832
              precision    recall  f1-score   support

       False       0.80      0.75      0.77       861
        True       0.77      0.82      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



## 9. Modelo 2: Random Forest

In [10]:
# Modelo 2 - Random Forest

from sklearn.ensemble import RandomForestClassifier

print("--- Treinando Modelo 2: Random Forest ---")
modelo_rf = RandomForestClassifier(random_state=42)
# Usando X_treino (NÃO normalizado)
modelo_rf.fit(X_treino, y_treino)

y_pred_rf = modelo_rf.predict(X_teste)

acuracia_rf = accuracy_score(y_teste, y_pred_rf)
print(f"\n--- Resultados do Random Forest (Base) ---")
print(f"Acurácia: {acuracia_rf:.4f}")
print(classification_report(y_teste, y_pred_rf))

--- Treinando Modelo 2: Random Forest ---

--- Resultados do Random Forest (Base) ---
Acurácia: 0.7872
              precision    recall  f1-score   support

       False       0.77      0.81      0.79       861
        True       0.80      0.76      0.78       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



## 10. Otimizando hiperparâmetros

In [11]:
# Otimização de Hiperparâmetros (GridSearchCV)

from sklearn.model_selection import GridSearchCV

print("--- Iniciando Otimização (GridSearchCV) ---")
print("AVISO: Isso pode demorar alguns minutos...")

# Parâmetros (simplificado para ser MAIS RÁPIDO)
parametros = {
    'n_estimators': [100],  # Apenas 100
    'max_depth': [10, 20]   # Apenas 2 profundidades
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=parametros,
                           cv=2, # cv=2 (MAIS RÁPIDO)
                           n_jobs=-1,
                           scoring='accuracy',
                           verbose=1)

grid_search.fit(X_treino, y_treino)

print("\nOtimização Concluída!")
print(f"Melhores Parâmetros: {grid_search.best_params_}")

melhor_rf = grid_search.best_estimator_
y_pred_rf_otimizado = melhor_rf.predict(X_teste)
acuracia_rf_otimizado = accuracy_score(y_teste, y_pred_rf_otimizado)

print(f"\n--- Resultados do Random Forest (OTIMIZADO) ---")
print(f"Acurácia Otimizada: {acuracia_rf_otimizado:.4f}")
print(classification_report(y_teste, y_pred_rf_otimizado))

--- Iniciando Otimização (GridSearchCV) ---
AVISO: Isso pode demorar alguns minutos...
Fitting 2 folds for each of 2 candidates, totalling 4 fits

Otimização Concluída!
Melhores Parâmetros: {'max_depth': 10, 'n_estimators': 100}

--- Resultados do Random Forest (OTIMIZADO) ---
Acurácia Otimizada: 0.7953
              precision    recall  f1-score   support

       False       0.81      0.77      0.79       861
        True       0.78      0.82      0.80       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



## 11. Respostas finais

In [ ]:
# Comparação Final e Respostas

print("--- Comparação dos Modelos ---")
# Vamos garantir que as variáveis existem caso algum bloco falhe
try:
    print(f"Acurácia Regressão Logística: {acuracia_lr:.4f}")
    print(f"Acurácia Random Forest (Base): {acuracia_rf:.4f}")
    print(f"Acurácia Random Forest (Otimizado): {acuracia_rf_otimizado:.4f}")
    melhor_acuracia = acuracia_rf_otimizado
except NameError:
    print("ERRO: Um dos modelos falhou em treinar. Não é possível comparar.")
    melhor_acuracia = 0 # Valor padrão para evitar mais erros

print("\n1. Meu modelo resolve adequadamente o problema proposto?")
print(f"Sim. O objetivo era prever se um passageiro seria 'Transportado' ou não. "
      f"Nosso modelo final (Random Forest Otimizado) alcançou uma acurácia de {melhor_acuracia:.2%}, "
      f"o que significa que ele acerta a previsão para quase {int(melhor_acuracia*100)} em cada 100 passageiros. "
      f"Isso é significativamente melhor do que um palpite aleatório (50%).")

print("\n2. Meu modelo pode ser colocado em produção?")
print(f"Sim, o modelo pode ser colocado em produção. O fluxo de 'pipeline' está bem definido: "
      f"os novos dados (passageiros) precisariam passar pelo *exatamente* mesmo tratamento "
      f"que fizemos (separação da 'Cabin', preenchimento de 'NaNs' com mediana/moda, e One-Hot Encoding) "
      f"antes de serem enviados ao modelo 'melhor_rf.predict()'. A acurácia de {melhor_acuracia:.2%} "
      f"é considerada robusta para este problema.")

--- Comparação dos Modelos ---
Acurácia Regressão Logística: 0.7832
Acurácia Random Forest (Base): 0.7872
Acurácia Random Forest (Otimizado): 0.7953


--- RESPOSTAS PARA O PROFESSOR ---

1. Meu modelo resolve adequadamente o problema proposto?
Sim. O objetivo era prever se um passageiro seria 'Transportado' ou não. Nosso modelo final (Random Forest Otimizado) alcançou uma acurácia de 79.53%, o que significa que ele acerta a previsão para quase 79 em cada 100 passageiros. Isso é significativamente melhor do que um palpite aleatório (50%).

2. Meu modelo pode ser colocado em produção?
Sim, o modelo pode ser colocado em produção. O fluxo de 'pipeline' está bem definido: os novos dados (passageiros) precisariam passar pelo *exatamente* mesmo tratamento que fizemos (separação da 'Cabin', preenchimento de 'NaNs' com mediana/moda, e One-Hot Encoding) antes de serem enviados ao modelo 'melhor_rf.predict()'. A acurácia de 79.53% é considerada robusta para este problema.


## 12. Conclusão

Este projeto teve como objetivo desenvolver um modelo de Machine Learning de ponta a ponta, utilizando o dataset "Spaceship Titanic" do Kaggle. O problema central era de classificação binária: prever se um passageiro seria "transportado" (a variável alvo Transported) com base em seus dados pessoais e de viagem.

O processo seguiu todas as etapas fundamentais de um fluxo de trabalho de ciência de dados:

1) Análise Exploratória e Limpeza de Dados: O dataset foi carregado e inspecionado. Identificamos uma quantidade significativa de valores ausentes (NaN) em colunas críticas como Age, HomePlanet, CryoSleep e Cabin. Estes valores foram tratados por imputação, usando a mediana para dados numéricos e a moda (valor mais frequente) para dados categóricos.

2) Engenharia de Features: A coluna Cabin foi processada e dividida em três novas features mais informativas: Deck, Num e Side, o que permitiu ao modelo capturar melhor a localização física dos passageiros. Colunas textuais irrelevantes para a previsão, como Name, PassengerId e Destination, foram removidas.

3) Pré-processamento: Todas as features categóricas restantes (HomePlanet, Deck, Side) foram transformadas em representações numéricas através do One-Hot Encoding. Os dados booleanos (CryoSleep, VIP) foram convertidos para inteiros (0/1).

4) Seleção de Métrica: A Acurácia (Accuracy) foi escolhida como a métrica principal de avaliação, pois o dataset apresentou um bom balanceamento entre as duas classes (transportados e não transportados).

Conforme solicitado nas instruções, dois estimadores foram comparados:

Modelo 1: Regressão Logística: Após a normalização dos dados com StandardScaler, este modelo serviu como nossa baseline, alcançando uma acurácia de 78.32%.

Modelo 2: Random Forest: Este modelo, baseado em árvores de decisão, obteve um resultado ligeiramente superior em sua forma base, com 78.72% de acurácia.

Para atender ao requisito de otimização, aplicamos um GridSearchCV ao Random Forest. Ao testar diferentes hiperparâmetros (max_depth e n_estimators), conseguimos encontrar uma configuração otimizada que elevou a performance final do modelo para 79.53% de acurácia nos dados de teste.

Este resultado confirma que o modelo de Random Forest Otimizado não apenas resolve o problema proposto com uma precisão significativamente superior à de um palpite aleatório, mas também estabelece um pipeline de tratamento de dados claro e replicável. Isso valida sua viabilidade para uma eventual implantação em produção, onde novos dados poderiam passar pelo mesmo processo de limpeza e transformação antes de serem enviados ao modelo para previsão.